In [1]:
import numpy as np 
import pandas as pd

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv("test.csv")

# Data

In [3]:
train_df.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train_df[train_df["target"] == 1].text.values[1]

'Forest fire near La Ronge Sask. Canada'

In [5]:
train_df[train_df["target"] == 0].text.values[1]

'I love fruits'

# Building vectors

In [6]:
from sklearn import feature_extraction

In [7]:
count_vectorizer =  feature_extraction.text.CountVectorizer()
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [8]:
print(example_train_vectors.todense().shape)
print(example_train_vectors.todense())

(5, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 2 0 1 1 1 0 0 0 0 1 1 0 0 0 0 0 2 0 0 0 0 0 1 1 0 1 1 1 1 0 0
  0 2 0 0 0 1 0 0 0 0 0 2 0 0 0 1 0 0]
 [1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1
  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
  1 0 1 0 0 0 0 1 0 1 1 0 1 0 1 0 0 1]]


Summary on above
1. There are 54 unique words from 5 tweets
2. Example first tweets contains only some of the unique words. The non zero values are the word that do exists in the first tweet

In [9]:
train_vectors = count_vectorizer.fit_transform(train_df['text'])
test_vectors = count_vectorizer.transform(test_df["text"])

Lets check how model behaves on training data using cross validation

In [10]:
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import cross_val_score

In [11]:
help(cross_val_score)

Help on function cross_val_score in module sklearn.model_selection._validation:

cross_val_score(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)
    Evaluate a score by cross-validation
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    estimator : estimator object implementing 'fit'
        The object to use to fit the data.
    
    X : array-like of shape (n_samples, n_features)
        The data to fit. Can be for example a list, or an array.
    
    y : array-like of shape (n_samples,) or (n_samples, n_outputs),             default=None
        The target variable to try to predict in the case of
        supervised learning.
    
    groups : array-like of shape (n_samples,), default=None
        Group labels for the samples used while splitting the dataset into
        train/test set. Only used in conjunction with a "Group" :term:`cv`

In [12]:
clf = RidgeClassifier()
scores = cross_val_score(clf,train_vectors,train_df['target'],cv=3,scoring= 'f1')

In [13]:
scores

array([0.59453669, 0.56498283, 0.64082434])

In [14]:
clf.fit(train_vectors,train_df['target'])

RidgeClassifier()

In [16]:
sub_df = pd.read_csv("sample_submission.csv")

In [17]:
sub_df['target'] = clf.predict(test_vectors)

In [18]:
sub_df.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [19]:
sub_df.to_csv("submission.csv",index=False)